### Importación de librerías

In [1]:
import cv2 as cv
from ultralytics import YOLO
import numpy as np

## Carga del Modelo y del Video a utilizar

In [2]:
# Cargamos el modelo preentrenado
model = YOLO('yolov8m.pt') 

In [3]:
# Aqui se abre el video 
cap = cv.VideoCapture('V_Mall.mp4')

## Configuraciones inciales

In [4]:
# Se configura la resolución del video capturado
cap.set(cv.CAP_PROP_FRAME_WIDTH, 1024)  # Aquí establecemos el ancho 
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 576)  # Aquí establecemos el alto

False

In [5]:
# Este es el umbral de confianza para las detecciones
confidence_threshold = 0.5

In [6]:
# Variables para poder rastrear posiciones y valores persistentes
previous_positions = {}
frame_count = 0
person_count = 0
density = 0
moving_ratio = 0
occupied_area_ratio = 0

In [7]:
# Configuración del VideoWriter
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

output_filename = 'video_resultado.avi'
fourcc = cv.VideoWriter_fourcc(*'XVID')  # Codec para AVI
out = cv.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

## Procesamiento del Video

In [8]:
# Procesar video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break

    # Realizar detección de objetos con YOLOv8
    results = model(frame)

    # Obtener las detecciones en formato xyxy
    detections = results[0].boxes

    # Variables para conteo y cálculo de densidad
    current_person_count = 0
    moving_count = 0
    static_count = 0
    total_box_area = 0
    current_positions = {}

    for det in detections:
        # Los resultados se almacenan como [x1, y1, x2, y2, conf, class_id]
        x1, y1, x2, y2 = det.xyxy[0]
        conf = det.conf[0]
        class_id = det.cls[0]

        if conf > confidence_threshold and class_id == 0:  # Clase 'person'
            # Incrementar el conteo de personas
            current_person_count += 1

            # Dibujar las cajas de las personas detectadas
            cv.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv.putText(frame, f'Person: {conf:.2f}', (int(x1), int(y1) - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Guardar la posición actual de la persona
            current_positions[current_person_count] = ((x1 + x2) / 2, (y1 + y2) / 2)

            # Calcular el área ocupada por la caja
            total_box_area += (x2 - x1) * (y2 - y1)

    # Actualizar valores solo si hubo detecciones
    if current_person_count > 0:
        person_count = current_person_count

        # Calcular densidad de personas
        frame_area = frame.shape[0] * frame.shape[1]
        density = person_count / frame_area if frame_area > 0 else 0

        # Determinar si las personas están en movimiento o estáticas
        for person_id, current_pos in current_positions.items():
            if person_id in previous_positions:
                prev_pos = previous_positions[person_id]
                distance = ((current_pos[0] - prev_pos[0]) ** 2 +
                            (current_pos[1] - prev_pos[1]) ** 2) ** 0.5
                if distance > 10:  # Umbral para considerar "en movimiento"
                    moving_count += 1
                else:
                    static_count += 1
            else:
                static_count += 1

        # Calcular proporción de movimiento
        total_detected = moving_count + static_count
        moving_ratio = moving_count / total_detected if total_detected > 0 else 0

        # Calcular porcentaje de área ocupada por las cajas
        occupied_area_ratio = total_box_area / frame_area if frame_area > 0 else 0

    # Actualizar posiciones para el siguiente frame
    previous_positions = current_positions

    # Cambiar color del texto basado en el conteo de personas
    text_color = (0, 255, 0) if person_count < 10 else (0, 0, 255)

    # Mostrar indicadores en el frame
    cv.putText(frame, f'Personas: {person_count}', (10, 40),
               cv.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
    cv.putText(frame, f'Densidad: {density:.6f}', (10, 80),
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv.putText(frame, f'Proporcion de Movimiento: {moving_ratio:.2f}', (10, 120),
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv.putText(frame, f'Area Ocupada: {occupied_area_ratio:.2%}', (10, 160),
               cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Redimensionar el frame antes de mostrarlo
    frame_resized = cv.resize(frame, (1024, 576))

    # Mostrar el frame con las detecciones
    cv.imshow('Detección en Vivo', frame_resized)

    # Mostrar el frame procesado
    frame_resized = cv.resize(frame, (1024, 576))
    cv.imshow('Detección en Vivo', frame_resized)

    # Escribir el frame procesado en el archivo de video
    out.write(frame_resized)

    # Salir si se presiona la tecla 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

# Liberar recursos y cerrar la ventana
cap.release()
cv.destroyAllWindows()


0: 384x640 11 persons, 1 handbag, 576.8ms
Speed: 18.6ms preprocess, 576.8ms inference, 32.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 handbags, 354.4ms
Speed: 27.9ms preprocess, 354.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 3 handbags, 411.4ms
Speed: 2.4ms preprocess, 411.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 5 handbags, 370.7ms
Speed: 3.0ms preprocess, 370.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 handbags, 550.0ms
Speed: 3.0ms preprocess, 550.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 3 handbags, 568.8ms
Speed: 4.5ms preprocess, 568.8ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 4 handbags, 627.9ms
Speed: 7.0ms preprocess, 627.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 